In [1]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re, os, random
import redis, pymysql
import time
from PIL import  Image
import cv2
import win32api,win32con
import numpy as np

In [2]:
#准备数据vv
def Ready_data():
 
        browser = webdriver.Chrome(r'C:\Users\Administrator\Downloads\chromedriver.exe')
        con = pymysql.connect(
            host='211.149.228.56',
            user='root',
            passwd='firstdb123',
            port=3306,
            db='firstdb',
            charset='utf8',
            cursorclass=pymysql.cursors.DictCursor
        )
        r = redis.Redis(host='localhost', port=6379)
        redis_data_dict_1 = "unused_id"
        id1 = bytes.decode(r.spop(redis_data_dict_1))
        print(id1)
        sql = 'select * from fbd_store WHERE id=%s' % id1
        cursor = con.cursor()
        cursor.execute(sql)
        result = cursor.fetchone()
        title = result['title']
        acreage = str(int(result['acreage']))
        total_price = str(int(result['total_price']))
        width = result['width'].strip('m')
        if width == '未填写'or '':
            width = '1'
        else:
            width = width
        depth = result['depth'].strip('m')
        if depth == '未填写'or '':
            depth = '1'
        else:
            depth = depth
        height = result['height'].strip('m')
        if height == '未填写'or '':
            height = '1'
        else:
            height = height
        address = result['address'].split(' ')
        if len(address) >=2:
            region = address[0].strip('区')
            local = address[1]
        else:
            region = address[0].strip('区')
            local = address[0].strip('区')
        store_status = result['store_status']
        pic_urls = result['pic_urls'].split(",")
        data_path = r"D:\\fangyuandata\\" + id1 + "\\"
        images_path = data_path + 'images\\'
        os.makedirs(images_path)
        for image_id in pic_urls:
            image_name = images_path+image_id+".jpg"
            sql = 'select img from pic_info WHERE id=%s' % image_id
            cursor.execute(sql)
            img = cursor.fetchone()
            with open(image_name,'wb') as f:
                f.write(img['img'])
        date = result['date'].split('-')
        if re.findall(r'共\d层', result['floor']):
            floor = re.findall(r'共(\d)层', result['floor'])
        else:
            floor = '1'
        content = result['intro']
        re_phone = re.compile(r'\d{11}')
        if re.search(re_phone, content) != None:
            print(re.search(re_phone, content).group())
            content = content.replace(str(re.search(re_phone, content).group()), '')
        info = content.replace('联系我时，请说是在58同城上看到的，谢谢！','')
        info_path = data_path+id1+'.html'
        with open (info_path,'w',encoding='utf-8') as f:
            f.write('<head>\n')
            f.write('    <meta charset="UTF-8">\n')
            f.write('    <title>Title</title>\n')
            f.write('</head>\n')
            f.write('<body>\n')
            f.write(info + '\n')
            f.write('<body>\n')
        otemp = r'C:\Users\Administrator\Downloads\captcha_img.jpg'
        oblk = r'C:\Users\Administrator\Downloads\captcha_img (1).jpg'
       
        return title,acreage,total_price,width,depth,height,region,local,store_status,pic_urls,images_path,info_path,floor,browser,otemp,oblk

    #用户登录
def Login(username,password,browser):
    browser.get('http://vip.anjuke.com/login/?errmsg=%E7%94%A8%E6%88%B7%E5%90%8D%E5%AF%86%E7%A0%81%E9%94%99%E8%AF%AF')
    try:
        print("-----------------------------------------------------------")
        print('正在输入账号.....')
        browser.find_element_by_xpath('//*[@id="loginName"]').send_keys(username)
        print('您的账号为%s' % username)
        time.sleep(1)
        print('正在输入密码.....')
        browser.find_element_by_xpath('//*[@id="loginPwd"]').send_keys(password)
        print('您的密码为%s' % password)
        time.sleep(1)
        browser.find_element_by_xpath('//*[@id="loginSubmit"]').click()
        print('正在登录......')
        time.sleep(1)
        print('登录成功')
    except Exception as e:
        print('登录失败')
        print(e)
        time.sleep(1)
        browser.quit()


#进入发布页面
def Enter_releasepage(info_path,browser):
    browser.find_element_by_xpath('//*[@id="qualification_check"]/div[1]/a').click()
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="huoketool_dialog"]/div[1]/a').click()
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="broker_nav"]/li[7]/a').click()
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="broker_nav"]/li[7]/dl/dt[1]/a').click()
    time.sleep(2)
    home_page = browser.current_window_handle
    windowhanles = browser.window_handles
    for handle in windowhanles:
        if handle != home_page:
            browser.switch_to_window(handle)
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="chooseWebForm"]/div[1]/label[2]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="chooseWebFormCommiter"]').click()
    time.sleep(2)
    browser.find_element_by_xpath('/html/body/div[4]/div[1]/a').click()
    time.sleep(2)
    windowhanles = browser.window_handles
    browser.switch_to_window(windowhanles[2])
    time.sleep(2)
    browser.get('file:///'+ info_path)
    time.sleep(2)
    browser.switch_to_window(windowhanles[1])
# 填写数据
def Input_data(title,acreage,total_price,width,
               depth,height,region,local,
               store_status,floor,browser):
    time.sleep(2)
    try:
        print("-----------------------------------------------------------")
        print('正在选择租售方式')
        browser.find_element_by_xpath('//*[@id="jp-sale"]').click()
        print('选择租售方式为出售')
    except Exception as e:
        print("##############################################")
        print('出现错误：%s' % e)
        print("##############################################")
        time.sleep(2)
        for i in range(5):
            time.sleep(1)
            print('正在第%s重新选择' %(i+1))
            browser.find_element_by_xpath('//*[@id="jp-sale"]').click()
            if True:
                break
    time.sleep(2)
    browser.find_element_by_xpath('//*[@id="current_status"]/label[2]').click()
    time.sleep(2)
    if browser.find_elements_by_xpath('//*[@id="select-jpType"]/div/div'):
            try:
                print("-----------------------------------------------------------")
                print('正在进入商铺类型选择')
                browser.find_element_by_xpath('//*[@id="select-jpType"]/div/div').click()
                print('进入成功')
            except Exception as e:
                print("##############################################")
                print('出现错误：%s' % e)
                print("##############################################")
                time.sleep(2)
                for i in range(5):
                    time.sleep(1)
                    print('正在第%s次重新进入' % (i+1))
                    browser.find_element_by_xpath('//*[@id="select-jpType"]/div/div').click()
                    if True:
                        break

            time.sleep(2)
            try:
                print("-----------------------------------------------------------")
                print('正在选择商铺类型')
                browser.find_element_by_xpath('//*[@id="select-jpType"]/div/ul/li[4]').click()
                print('选择商铺类型为社区底商')
            except Exception as e:
                print("##############################################")
                print('出现错误：%s' % e)
                print("##############################################")
                time.sleep(2)
                for i in range(5):
                    time.sleep(1)
                    print('正在第%s重新选择' %(i+1))
                    browser.find_element_by_xpath('//*[@id="select-jpType"]/div/ul/li[4]').click()
                    if True:
                        break

            time.sleep(2)
            try:
                print("-----------------------------------------------------------")
                print('正在进入58商铺性质选择')
                browser.find_element_by_xpath('//*[@id="select-jpProperty"]/div/div').click()
                print('进入成功')
            except Exception as e:
                print("##############################################")
                print('出现错误：%s' % e)
                print("##############################################")
                for i in range(5):
                    time.sleep(2)
                    print('正在第%s次重新进入' % (i+1))
                    browser.find_element_by_xpath('//*[@id="select-jpProperty"]/div/div').click()
                    if True:
                        print("重新选择成功")
                        break

            time.sleep(2)
            try:
                print("-----------------------------------------------------------")
                print('正在选择商铺性质')
                browser.find_element_by_xpath('//*[@id="select-jpProperty"]/div/ul/li[3]').click()
                print('选择商铺性质为商铺新房')
            except Exception as e:
                print("##############################################")
                print('出现错误：%s' % e)
                print("##############################################")
                for i in range(5):
                    time.sleep(2)
                    print('正在第%s次重新进入' %(i+1))
                    browser.find_element_by_xpath('//*[@id="select-jpProperty"]/div/ul/li[3]').click()
                    if True:
                        break

    time.sleep(1)
# 商铺所在层数
    browser.find_element_by_name('startFloor').send_keys('1')
    time.sleep(1)
# 总层数
    browser.find_element_by_name('totalFloor').send_keys(floor)
    time.sleep(1)
# 总面积
    browser.find_element_by_name('roomarea').send_keys(acreage)
    time.sleep(1)
# 面宽
    browser.find_element_by_name('faceWidth').send_keys(width)
    time.sleep(1)
# 层高
    browser.find_element_by_name('floorHigh').send_keys(height)
    time.sleep(1)
# 进深
    browser.find_element_by_name('spatialDepth').send_keys(depth)
    time.sleep(1)

    for i in browser.find_elements_by_name('fitment[]'):
        time.sleep(0.5)
        i.click()

    for i in (browser.find_elements_by_name('customer[]')[:3]):
        time.sleep(0.5)
        i.click()

    time.sleep(1)
    browser.find_element_by_xpath("//*[text()='是']").click()
    
    time.sleep(3)
    if browser.find_elements_by_xpath('//*[@id="ajk-jp-input"]'):
        try:
            print("-----------------------------------------------------------")
            print('正在填写安居客地址')
            print("-----------------------------------------------------------")

            browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').send_keys(local)
            time.sleep(3)
            if browser.find_elements_by_xpath('//*[@id="publish_form"]/div[10]/div/ul/li'):
                print('填写地址为：%s' % (local))
            else:
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').clear()
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').send_keys(region)
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').click()
                time.sleep(3)
                if browser.find_elements_by_xpath('//*[@id="publish_form"]/div[13]/div/ul/li'):
                    print('填写地址为：%s' % region)

        except Exception as e:
            print("##############################################")
            print('出现错误：%s' % e)
            print("##############################################")
            for i in range(5):
                print('正在第%s次重新填写' % (i+1))
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').clear()
                time.sleep(1)
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').send_keys(region)
                time.sleep(1)
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').send_keys(local)
                time.sleep(1)
                if browser.find_elements_by_xpath('//*[@id="publish_form"]/div[10]/div/ul/li'):
                    browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').click()
                    print('填写地址为：%s' % (region+local))
                else:
                    browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').clear()
                    browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').send_keys(region)
                    print('填写地址为：%s' % region)
                    if True:
                        print('填写成功')
                        break

        time.sleep(2)
        try:
            # browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').click()
            print("-----------------------------------------------------------")
            print('正在选择安居客地址')
            print("-----------------------------------------------------------")
            # time.sleep(2)
            # browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').click()
            time.sleep(3)
            browser.find_element_by_xpath('//*[@id="publish_form"]/div[10]/div/ul/li[%s]' % (random.randint(1,len(browser.find_elements_by_xpath('//*[@id="publish_form"]/div[10]/div/ul/li'))))).click()
        except Exception as e:
            print("##############################################")
            print('出现错误：%s' % e)
            print("##############################################")
            for i in range(5):
                time.sleep(2)
                print('正在第%s次重新选择' % (i+1))
                browser.find_element_by_xpath('//*[@id="detail_address58"]').click()
                time.sleep(1)
                browser.find_element_by_xpath('//*[@id="ajk-jp-input"]').click()
                time.sleep(3)
                browser.find_element_by_xpath('//*[@id="publish_form"]/div[10and]/div/ul/li[%s]' % (random.randint(1,len(browser.find_elements_by_xpath('//*[@id="publish_form"]/div[10]/div/ul/li'))))).click()
                if True:
                    print('选择成功')
                    break
        try:
            browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/div').click()
            time.sleep(1)
            browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
            time.sleep(1)
            browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/div').click()
            try:
                f = browser.find_elements_by_xpath('//*[@id="select-zoneAJK"]/div/ul/li')
                for i in f:
                    if i.text == region:
                        print('安居客区域为：%s' % region)
                        i.click()
            except:
                browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/div').click()
                time.sleep(1)
                browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
                time.sleep(1)
                browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/div').click()
                time.sleep(1)
                f = browser.find_elements_by_xpath('//*[@id="select-zoneAJK"]/div/ul/li')
                for i in range(5):
                    print('正在第%s重新选择' % (i+1))
                    for i in f:
                        if i.text == region:
                            print('安居客区域为：%s' % region)
                            i.click()
                    browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/ul/li[8]').click()
        except:
            print('无此选项')

        try:
            browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
            time.sleep(1)
            browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/div').click()
            time.sleep(1)
            browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
            time.sleep(1)
            try:
                f = browser.find_elements_by_xpath('//*[@id="select-blockAJK"]/div/ul/li')
                for i in f:
                    if i.text == local:
                        print('安居客地点为：%s' % local)
                        i.click()
                # browser.find_element_by_xpath('//*[@id="select-blockGJ"]/div/ul/li[1]').click()
            except:
                browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
                browser.find_element_by_xpath('//*[@id="select-blockAJK"]/div/div').click()
                f = browser.find_elements_by_xpath('//*[@id="select-blockAJK"]/div/ul/li')
                for i in range(5):
                    print('正在第%s重新选择' % (i+1))
                    for i in f:
                        if i.text == region:
                            print('安居客区域为：%s' % local)
                            i.click()
                    # browser.find_element_by_xpath('//*[@id="select-zoneAJK"]/div/ul/li[8]').click()
        except:
            print('无此选项')
    else:
        print('无安居客选项')
# 58位置信息
    if browser.find_elements_by_xpath('//*[@id="detail_address58"]'):
        try:
            print("-----------------------------------------------------------")
            print('正在填写58地址')
            browser.find_element_by_xpath('//*[@id="detail_address58"]').send_keys(region)
            time.sleep(1)
            browser.find_element_by_xpath('//*[@id="detail_address58"]').send_keys(local)
            time.sleep(3)
            if browser.find_elements_by_xpath('//*[@id="publish_form"]/div[16]/div/ul/li'):
                print('填写地址为：%s' % (region+local))
            else:
                browser.find_element_by_xpath('//*[@id="detail_address58"]').clear()
                browser.find_element_by_xpath('//*[@id="detail_address58"]').send_keys(local)
                browser.find_element_by_xpath('//*[@id="detail_address58"]').click()
        except Exception as e:
            print("##############################################")
            print('出现错误：%s' % e)
            print("##############################################")
            for i in range(1,5):
                print('正在第%s次重新填写' % i)
                browser.find_element_by_xpath('//*[@id="detail_address58"]').clear()
                time.sleep(2)
                browser.find_element_by_xpath('//*[@id="detail_address58"]').send_keys(local)
                if True:
                    print('填写成功')
                    break

        try:
            time.sleep(3)
            print("-----------------------------------------------------------")
            print('正在选择58地址')
            locator = (By.XPATH, '//*[@id="publish_form"]/div[16]/div/ul/li')
            WebdriverWait(browser,10,1).until(EC.presence_of_all_elements_located(locator))
            browser.find_element_by_xpath('//*[@id="publish_form"]/div[16]/div/ul/li[%s]' % random.randint(1, len(browser.find_elements_by_xpath('//*[@id="publish_form"]/div[16]/div/ul/li')))).click()
        except Exception as e:
            print("##############################################")
            print('出现错误：%s' % e)
            print("##############################################")
            for i  in range(5):
                time.sleep(2)
                print('正在第%s次重新选择' % (i+1))
                browser.find_element_by_xpath('//*[@id="detail_address58"]').click()
                time.sleep(2)
                locator = (By.XPATH, '//*[@id="publish_form"]/div[16]/div/ul/li')
                WebDriverWait(browser, 10, 1).until(EC.presence_of_all_elements_located(locator))
                browser.find_element_by_xpath('//*[@id="publish_form"]/div[16]/div/ul/li[%s]' % random.randint(1, len(browser.find_elements_by_xpath('//*[@id="publish_form"]/div[16]/div/ul/li')))).click()
                if True:
                    print('选择成功')
                    break
# 58区域选择
        try:
            time.sleep(2)
            browser.find_element_by_xpath('//*[@id="select-zone58"]/div/div').click()
            time.sleep(2)
            f = browser.find_elements_by_xpath('//*[@id="select-zone58"]/div/ul/li')
            time.sleep(1)
            for i in f:
                if i.text == region:
                    print('58区域为：%s' % region)
                    i.click()
                    break
                elif i.text == (region + "区"):
                    print('58区域为：%s' % region)
                    i.click()
        except Exception as e:
            print(e)
            for i in range(5):
                print('正在第%s次从选择')
                time.sleep(2)
                browser.find_element_by_xpath('//*[@id="select-zone58"]/div/div').click()
                time.sleep(2)
                f = browser.find_elements_by_xpath('//*[@id="select-zone58"]/div/ul/li')
                time.sleep(1)
                for i in f:
                    if i.text == region:
                        print('58区域为：%s' % region)
                        i.click()
                        if True:
                            break
                    elif i.text == (region + "区"):
                        print('58区域：%s' % region)
                        i.click()
        try:
            time.sleep(2)
            browser.find_element_by_xpath('//*[@id="select-block58"]/div/div').click()
            time.sleep(2)
            f = browser.find_elements_by_xpath('//*[@id="select-block58"]/div/ul/li')
            time.sleep(1)
            for i in f:
                if i.text == local:
                    print('58地点为：%s' % local)
                    i.click()
                    break
        except Exception as e:
            print(e)
            for i in range(5):
                print('正在第%s次从选择')
                time.sleep(2)
                browser.find_element_by_xpath('//*[@id="select-block58"]/div/div').click()
                time.sleep(2)
                f = browser.find_elements_by_xpath('//*[@id="select-block58"]/div/ul/li')
                time.sleep(1)
                for i in f:
                    if i.text == local:
                        print('58地点为：%s' % local)
                        i.click()
                        if True:
                            break

# 价格
    browser.find_element_by_xpath('//*[@id="saleprice-box-c"]/input').send_keys(total_price)

#填入描述
def Input_info(title,info_path,browser):
    
    browser.find_element_by_name('title').send_keys(title)
    time.sleep(1)
    
    windowhanles = browser.window_handles
    browser.switch_to_window(windowhanles[2])
    time.sleep(2)

    browser.find_element_by_tag_name('body').click()
    time.sleep(2)

    browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL+'a')
    time.sleep(2)

    browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL+'c')
    time.sleep(2)

    windowhanles = browser.window_handles
    time.sleep(2)

    browser.switch_to_window(windowhanles[1])
    time.sleep(2)

    browser.switch_to_frame(browser.find_element_by_xpath('//*[@id="editor-wrap"]/div[2]/div/div[2]/iframe').send_keys(Keys.CONTROL+'v'))

#上传图片
def Upload_images(images_path,browser):
    for i in os.listdir(images_path):
        try:
            image = images_path + i
            browser.find_element_by_name('file').send_keys(image)
            time.sleep(1)
        except:
            browser.find_element_by_xpath('//*[@id="alert"]/div[2]/a').click()

    for j in range(1,len(browser.find_elements_by_xpath('//*[@id="pic-upload-display"]/div'))):
        if j < 5:
            browser.find_element_by_xpath('//*[@id="pic-upload-display"]/div[%d]/div[1]/select/option[%d]' % (j, j + 1)).click()
        else:
            browser.find_element_by_xpath('//*[@id="pic-upload-display"]/div[%d]/div[1]/select/option[%d]' % (j, 4)).click()
    browser.find_element_by_xpath('//*[@id="pic-upload-display"]/div[%s]/div[2]/i[1]' % (random.randint(1, j))).click()
    
def download (browser):
    element1 = '//*[@id="ISDCaptcha"]/div[1]/div/img[1]'
    element2 = '//*[@id="ISDCaptcha"]/div[1]/div/img[2]'
    elements = [element1,element2]
    for element in elements:
        ActionChains(browser).context_click(browser.find_element_by_xpath(element)).perform()
        time.sleep(2)
        win32api.keybd_event(86, 0, 0, 0)
        win32api.keybd_event(86, 0, win32con.KEYEVENTF_KEYUP, 0)
        time.sleep(1)
        win32api.keybd_event(13, 0, 0, 0)
        win32api.keybd_event(13, 0, win32con.KEYEVENTF_KEYUP, 0)
        time.sleep(1)
    

def get_distance(browser,otemp,oblk):
    download(browser)
    target = cv2.imread(otemp, 0)
    template = cv2.imread(oblk, 0)
    w, h = target.shape[::-1]
    temp = 'temp.jpg'
    targ = 'targ.jpg'
    cv2.imwrite(temp, template)
    cv2.imwrite(targ, target)
    target = cv2.imread(targ)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    target = abs(255 - target)
    cv2.imwrite(targ, target)
    target = cv2.imread(targ)
    template = cv2.imread(temp)
    result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)
    x, y = np.unravel_index(result.argmax(), result.shape)
    # 展示圈出来的区域
#     cv2.rectangle(template, (y, x), (y + w, x + h), (7, 249, 151), 2)
    distance = round(y*280/480)
    return distance

def get_track(distance):
    '''
    拿到移动轨迹，模仿人的滑动行为，先匀加速后匀减速
    匀变速运动基本公式：
    ①v=v0+at
    ②s=v0t+(1/2)at²
    ③v²-v0²=2as

    :param distance: 需要移动的距离
    :return: 存放每0.2秒移动的距离
    '''
    tracks = []
    while True:
        x = random.randint(1,4)
        tracks.append(x)
        if sum(tracks)==distance:
            break
        elif (distance-sum(tracks))<x:
            tracks.append(distance-sum(tracks))
            break
    return tracks

def main_check_code(browser,otemp,oblk):
    """
     拖动识别验证码
    :param driver: 
    :param element: 
    :return: 
    """
    # 2 对比两张图片的所有RBG像素点，得到不一样像素点的x值，即要移动的距离
    distance = get_distance(browser,otemp,oblk)
    print('l=',distance)
    # 3 获得移动轨迹
    x_track_list = get_track(distance)
    y_track_list = [random.randint(-3,3) for i in range(len(x_track_list))]
    print('第一步,点击滑动按钮')
    ActionChains(browser).click_and_hold(on_element=browser.find_element_by_xpath('//*[@id="ISDCaptcha"]/div[2]/div[3]')).perform()  # 点击鼠标左键，按住不放
    time.sleep(1)
    print('第二步,拖动元素')
    for x_track,y_track in zip(x_track_list,y_track_list):
        ActionChains(browser).move_by_offset(xoffset=x_track, yoffset=y_track).perform()  # 鼠标移动到距离当前位置（x,y）
        time.sleep(random.randint(19,23)/1000)
    # if l>100:
    ActionChains(browser).move_by_offset(xoffset=-random.randint(1,2), yoffset=0).perform()
    time.sleep(1)
    print('第三步,释放鼠标')
    ActionChains(browser).release(on_element=browser.find_element_by_xpath('//*[@id="ISDCaptcha"]/div[2]/div[3]')).perform()
    time.sleep(5)
    status = True
    try:
        browser.find_element_by_xpath('//*[@id="broker_nav"]/li[7]/a')
        return status
    except:
        status = False
        return status

def isElement(browser): 
    browser = browser
    status = True
    try:
        browser.find_element_by_xpath('//*[@id="ISDCaptcha"]/div[2]')
        return status
    except:
        status = False
        return status

def Delete_image(otemp,oblk):
    os.remove(otemp)
    os.remove(oblk)

In [3]:
def run(username,password):
    try:
        title,acreage,total_price,width,depth,height,region,local,store_status,pic_urls,images_path,info_path,floor,browser,otemp,oblk = Ready_data() 

        Login(username,password ,browser)
        time.sleep(3)

        if isElement(browser)==True:
            if main_check_code(browser,otemp,oblk)==True:
                print('通过验证码')
                Delete_image(otemp,oblk)
            else:
                for i in range(5):
                    print('正在第%s次重试验证码' %(i))
                    Delete_image(otemp,oblk)
                    browser.find_element_by_xpath('//*[@id="ISDCaptcha"]/div[1]/div/div/div').click()
                    time.sleep(2)
                    if main_check_code(browser,otemp,oblk)== True:
                        print("第%s次成功破解" %(i))
                        Delete_image(otemp,oblk)
                        break
        else:
            print('未出现滑动验证码')
        Enter_releasepage(info_path,browser)
        Input_data(title,acreage,total_price,width,
                       depth,height,region,local,
                       store_status,floor,browser)
        Input_info(title,info_path,browser)
        Upload_images(images_path,browser)
        browser.find_element_by_xpath('//*[@id="publish-jpshop-add"]').click()
        print('发布成功')
        time.sleep(2)
        return True
    except:
        browser.quit()
          
    finally:
        browser.quit()
          

In [4]:
def main():
    agent_user = [['15760356617','baobao384659..',0]]
    
    while True:
        if len(agent_user)==0:
            break
        for user in agent_user:
            if user[2]<10:
                if run(user[0],user[1]) == True:
                    user[2] += 1
            else:
                continue
                agent_user.remove(user)
    

In [ ]:
main()

24954
-----------------------------------------------------------
正在输入账号.....
您的账号为15760356617
正在输入密码.....
您的密码为baobao384659..
正在登录......
登录成功
未出现滑动验证码


C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:122: DeprecationWarning: use driver.switch_to.window instead
C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:131: DeprecationWarning: use driver.switch_to.window instead
C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:135: DeprecationWarning: use driver.switch_to.window instead


-----------------------------------------------------------
正在选择租售方式
选择租售方式为出售
-----------------------------------------------------------
正在填写安居客地址
-----------------------------------------------------------
填写地址为：南延线
-----------------------------------------------------------
正在选择安居客地址
-----------------------------------------------------------
无此选项
无此选项


C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:508: DeprecationWarning: use driver.switch_to.window instead
C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:523: DeprecationWarning: use driver.switch_to.window instead
C:\anaconda3\envs\AI\lib\site-packages\ipykernel_launcher.py:526: DeprecationWarning: use driver.switch_to.frame instead


27178
-----------------------------------------------------------
正在输入账号.....
您的账号为15760356617
正在输入密码.....
您的密码为baobao384659..
正在登录......
登录成功
未出现滑动验证码
-----------------------------------------------------------
正在选择租售方式
选择租售方式为出售
-----------------------------------------------------------
正在填写安居客地址
-----------------------------------------------------------
填写地址为：蜀汉路
-----------------------------------------------------------
正在选择安居客地址
-----------------------------------------------------------
无此选项
无此选项
发布成功
25419
-----------------------------------------------------------
正在输入账号.....
您的账号为15760356617
正在输入密码.....
您的密码为baobao384659..
正在登录......
登录成功
未出现滑动验证码
-----------------------------------------------------------
正在选择租售方式
选择租售方式为出售
-----------------------------------------------------------
正在填写安居客地址
-----------------------------------------------------------
填写地址为：茶店子
-----------------------------------------------------------
正在选择安居客地址
-------------------------------------------